In [36]:
import pandas as pd
from Models import BertBaseline
import tensorflow as tf
import numpy as np
import re

In [3]:
df = pd.read_csv('../data/emrqa_tfidf_evidence_test.csv')
model = BertBaseline()

2024-04-10 21:06:35.347389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 21:06:35.381533: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 21:06:35.381723: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                        

In [4]:
model_dir = '../models/weights_Apr_10_bert_evidence_baseline'

In [5]:
mm = model.model

In [6]:
checkpoint = tf.train.Checkpoint(model=mm)
checkpoint.restore(tf.train.latest_checkpoint(model_dir))

In [7]:
df.iloc[0]

question           what is the dosage of the medication the patie...
tf-idf_evidence    after the\nuse of beta blockade and adenosine ...
answer                                                     adenosine
start_token                                                       27
end_token                                                         29
Name: 0, dtype: object

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [9]:
a = tokenizer(df['question'].to_list(), df['tf-idf_evidence'].to_list(), padding=True, truncation=True, return_tensors='tf')

In [10]:
predictions = mm.predict((a['input_ids'], a['attention_mask']))

207/207 [==============================] - 102s 487ms/step


In [11]:
starts = []
ends = []
for i in predictions:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
predictions

,start,end
0,0,22
1,0,45
2,106,59
3,0,10
4,1,13
...,...,...
6596,64,21
6597,0,23
6598,38,53
6599,17,27


In [17]:
df['predicted_start'], df['predicted_end'] = starts, ends

In [12]:
answers = df['answer'].to_list()
answers[:4]

['adenosine', '150', '1-2 tab', 'a ct scan of the head']

In [13]:
len("".split())

0

In [18]:
tokenizer.decode()

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [28]:
tokenizer.decode(tokenizer(df['tf-idf_evidence'][0], padding=True, truncation=True)['input_ids'][df['predicted_start'][0]:df['predicted_end'][0]])

'[CLS] after the use of beta blockade and adenosine, her arrhythmia resolved. [SEP]'

In [29]:
def get_prediction(evidence, predicted_start, predicted_end):
    tokenized_evidence = tokenizer(evidence, padding=True, truncation=True)['input_ids']
    predicted_tokens = tokenized_evidence[predicted_start:predicted_end]
    return tokenizer.decode(predicted_tokens)

In [39]:
def replace_punctuation(s):
    s = re.sub(r'[^\w\s]', '', s)
    s = " ".join(s.split())
    return s

def exact_match(prediction, truth):
    return int(prediction.split() == truth.split())

def shared_words(prediction, truth):
    # find shared words between prediction and truth while
    a = set(prediction.split())
    b = set(truth.split())
    return a & b

def precision(prediction, truth):
    num = len(shared_words(prediction, truth))
    den = len(truth.split())
    if num == 0 or den == 0:
        return 0
    return num/den

def recall(prediction, truth):
    num = len(shared_words(prediction, truth))
    den = len(prediction.split())
    if num == 0 or den == 0:
        return 0
    return num/den

def f1_score(prediction, truth):
    p = precision(prediction, truth)
    r = recall(prediction, truth)
    if p + r == 0:
        return 0
    return 2*p*r/(p+r)

def compute_metrics(prediction, truth):
    prediction = replace_punctuation(prediction)
    truth = replace_punctuation(truth)
    return exact_match(prediction, truth), precision(prediction, truth), recall(prediction, truth), f1_score(prediction, truth)

In [40]:
compute_metrics(get_prediction(df['tf-idf_evidence'][0], df['predicted_start'][0], df['predicted_end'][0]), df['answer'][0])

(0, 1.0, 0.07692307692307693, 0.14285714285714288)

In [41]:
df['predictions'] = df.apply(lambda x: get_prediction(x['tf-idf_evidence'], x['predicted_start'], x['predicted_end']), axis=1)
df['exact_match'], df['precision'], df['recall'], df['f1_score'] = zip(*df.apply(lambda x: compute_metrics(x['predictions'], x['answer']), axis=1))
# get average metrics
df[['exact_match', 'precision', 'recall', 'f1_score']].mean()

exact_match    0.000303
precision      0.380667
recall         0.062439
f1_score       0.097360
dtype: float64

In [47]:
# calculate how many were unanswerable, this is if start_token and end_token are both 0
df['unanswerable'] = df.apply(lambda x: int(x['start_token'] == 0 and x['end_token'] == 0), axis=1)
df['unanswerable'].mean()

0.4809877291319497

In [48]:
# calculate metrics for samples that are answerable
df[df['unanswerable'] == 0][['exact_match', 'precision', 'recall', 'f1_score']].mean()

exact_match    0.000292
precision      0.600228
recall         0.100859
f1_score       0.157773
dtype: float64